In [1]:
import pandas as pd
from TwitterSearch import *

In [2]:
KEY_WORDS = ('сбербанк', 'sberbank', 'втб', 'vtb', 'газпромбанк', 'gazprombank', 'альфабанк', 'alfabank',
             'райффайзен', 'raiffeisen', 'райффайзенбанк', 'raiffeisenbank', 'уралсиб', 'uralsib',
             'рсхб', 'rshb', 'россельхозбанк',
             'rshbmedia', 'raiffeisen_ru', 'uralsib_krd')

bases = set()
texts = list()
cur_tweet_id = 10e30
min_tweet_id = 0
max_tweet_id = int(cur_tweet_id)
response_id = 0

ts = TwitterSearch(
            consumer_key = '9pZzMqCZss66blrPuMRqsUVxz',
            consumer_secret = 'TSiOrPqKDZWi5G7AEVUpHh4eKQJ7j8IHrkv08aN3vr9L7JiciS',
            access_token = '4558856717-jRUboGPQzgHn83ikdSIX5p6a4rkFimPxbOiTzbm',
            access_token_secret = 'cmp8KIF0zIgJzD7AfZdeJiauUNWnoqR8VtwdDB2eAyyUK'
         )

In [3]:
def contains_keyword(text, key_words):
    text_lower = text.lower()
    for w in key_words:
        if w in text_lower:
            return True
    return False

def extract_base(text):
    tokens = text.strip().split()
    tokens = filter(lambda t: not (t.startswith('@') or t.startswith('#') or t.startswith('https://t.co') or t == 'RT'), tokens)
    basis = ' '.join(tokens)
    return basis

In [4]:
df = pd.read_excel('8390.xls', encoding='utf-8')
min_tweet_id = df.tweet_id.max()
bases = set(map(extract_base, df.text.values))

In [5]:
while True:
    try:
        tso = TwitterSearchOrder()

        # define all words we would like to have a look for
        tso.set_keywords(KEY_WORDS, or_operator=True) 
        tso.set_language('ru') # we want to see Russian tweets only
        tso.set_include_entities(False)
        tso.set_result_type('recent')
        tso.set_max_id(max_tweet_id)
        
        response = ts.search_tweets(tso)

        # check if there are statuses returned and whether we still have work to do
        if len(response['content']['statuses']) == 0:
            break

        # check all tweets according to their ID
        for tweet in response['content']['statuses']:
            cur_tweet_id = tweet['id']
            max_tweet_id = min(cur_tweet_id, max_tweet_id) - 1
            text = tweet['text']
            
            if not contains_keyword(text, map(lambda s: unicode(s.decode('utf-8')), KEY_WORDS)):
                continue
            base = extract_base(text)
            if base in bases:
                continue
            bases.add(base)
            texts.append([cur_tweet_id, text])
            
        print 'Request #{0}, texts {1}'.format(response_id, len(texts))
        
        response_id += 1
        

    except TwitterSearchException as e: # take care of all those ugly errors if there are some
        print(e)
        break
print "Total catch:", len(texts)

Request #0, texts 63
Request #1, texts 119
Request #2, texts 164
Request #3, texts 210
Request #4, texts 255
Request #5, texts 296
Request #6, texts 329
Request #7, texts 389
Request #8, texts 450
Request #9, texts 502
Request #10, texts 555
Request #11, texts 595
Request #12, texts 622
Request #13, texts 655
Request #14, texts 705
Request #15, texts 760
Request #16, texts 802
Request #17, texts 844
Request #18, texts 882
Request #19, texts 926
Request #20, texts 971
Request #21, texts 1013
Request #22, texts 1046
Request #23, texts 1074
Request #24, texts 1097
Request #25, texts 1118
Request #26, texts 1159
Request #27, texts 1202
Request #28, texts 1233
Request #29, texts 1259
Request #30, texts 1294
Request #31, texts 1335
Request #32, texts 1383
Request #33, texts 1437
Request #34, texts 1497
Request #35, texts 1546
Request #36, texts 1576
Request #37, texts 1608
Request #38, texts 1641
Request #39, texts 1685
Request #40, texts 1728
Request #41, texts 1775
Request #42, texts 1810


In [6]:
df_new = pd.DataFrame(texts, columns=['tweet_id', 'text'])
df_new.head()

,tweet_id,text
0,820220678284705796,"@Alex_Shvarz @alfabank Чему удивляться, бывает..."
1,820219628957876225,Эти роботы #Сбербанка - не переодетый #Греф? Б...
2,820219469146583040,I'm at ВТБ Ледовый дворец - @dynamo_ru for Рус...
3,820219002257633280,Результат пятничного деплоя от @sberbank https...
4,820218632282271744,RT @nstarikov: Сбербанк объявил о сокращении 3...


In [7]:
for text in texts[:100]:
    print text[1]
    print

@Alex_Shvarz @alfabank Чему удивляться, бывает и в три ночи звонки, страху нагоняют

Эти роботы #Сбербанка - не переодетый #Греф? Будут ли эти роботы предъявлять иски к #Сбербанк от имени клиентов и писать жалобы в ЦБ РФ?

I'm at ВТБ Ледовый дворец - @dynamo_ru for Русская Классика. Динамо Бшх - Химик in Москва w/ @bskkvmarina https://t.co/pHqO5JVII6

Результат пятничного деплоя от @sberbank https://t.co/7qbUqCN6yf

RT @nstarikov: Сбербанк объявил о сокращении 3000 юристов и замене их роботами. А вот Грефа менять не надо - он уже робот, судя по его отно…

Сбербанк профинансировал лизинг в Татарстане в рамках первой...

RT @Suerya14: Tonton di #Periscope: Скиньтесь мне на котика Карта Сбербанк 67628038 9783019222 https://t.co/yxmz9MYbJp

Смотрите в #Periscope: Скиньтесь мне на котика Карта Сбербанк 67628038 9783019222 https://t.co/jTg6WOI6Pv https://t.co/MkuRfh6x1N

Когда услышал, что Сбербанк закрывается horse with a blister Хорст вессель 206 было

Когда услышал, что Сбербанк закрывает

In [8]:
df_new.loc[:, 'tweet_id'] = df_new.tweet_id.astype(str)

In [9]:
df.loc[:, 'tweet_id'] = df.tweet_id.astype(str)
df_concat = pd.concat([df, df_new])
df_concat.index = range(df_concat.shape[0])
df_concat.shape

(12080, 3)

In [10]:
df_concat = df_concat[['tweet_id', 'text', 'label']]

In [11]:
df_concat.to_excel('12080.xls', index=False, encoding='utf-8')

---

# Labels

In [43]:
df = pd.read_excel('5221.xls', encoding='utf-8')
df.head()

,tweet_id,text,label
0,814131630948450304,@alfabank недели две имеет значение?,0
1,814131371690131456,@alfabank ну зачем же вы мне опять открытку на...,NaN
2,814130811784101888,@AsvHelp Сегодня 28.12.2016 из ВТБ 24 почтой...,0
3,814130709233332228,С вкладчиками Межрегионального Клирингового Ба...,0
4,814130458560606208,Сбербанк в 2016 году зафиксировал 74 кибератак...,0


In [44]:
df.label.value_counts()

 0    113
-1     27
 1      6
dtype: int64

In [49]:
df.ix[204]

tweet_id                    814070779474247680
text        о да, детка, сбербанк - onelove 💚
label                                      NaN
Name: 204, dtype: object

# TODO

In [55]:
print u'&amp;quot;Барис&amp;quot;'
print 'tinkoff_bank', 'сбер'

&amp;quot;Барис&amp;quot;
tinkoff_bank сбер
